In [2]:
import dbt
import json
import IPython.display as ipd
import utilities
import Segment
# import librosa
import Alignment
import librosa

In [13]:
song1_name = "despacito.wav"
# segmentino_name = song1_name.replace(".wav", "") + "_segmentino.txt"
# song1_data, sr = dbt.load_and_resample(song1_name)
# def create_segment_audio(data, name, segment_name, sr):
#     segments = Segment.extract_segment(segment_name)
#     start = int(segments[idx].start * sr)
#     end = int(segments[idx].end * sr)
#     utilities.slice_song_and_save(data, start, end, sr, name, str(idx))

# create_segment_audio(song1_data, song1_name, segmentino_name, sr)

In [5]:
print(len(song1_data) / sr)

237.68816326530612


In [14]:
json_song1_name = song1_name.replace(".wav", "") + ".json"
with open(json_song1_name, "w") as json_file:
    dump_data = []
#     song1_segments = Segment.extract_segment(segmentino_name)
    song_data, sr  = dbt.load_and_resample(song1_name)
    song_data = librosa.effects.harmonic(song_data)
    song1_segments = [Segment.Segment(0, len(song_data) / sr, "")]
    for idx in range(len(song1_segments)):
#         song_data,sr = dbt.load_and_resample(song1_name.replace(".wav", "") + "_" + str(idx) + ".wav")
        print("Loaded song_data with sr", sr, "length", len(song_data) / sr)
        print("Segment info:", song1_segments[idx].toString(), "idx", idx)    
        #Get Downbeats
        downbeats = dbt.extract_downbeat(4,song_data)
        
        time_1, beat_num1 = downbeats[:,0].tolist(), downbeats[:,1].tolist()
#         if time_1[0] > 0.07:
#             time_1 = [0.0] + time_1
#             beat_num1 = [(beat_num1[0] - 1) % 4] + beat_num1
        
        print("length time_1", len(time_1))
        #Get Chord
        chord_dict, chord_result1 = dbt.get_chords_old(time_1, song_data, sr, False)
        #Smoothen the chords
        print("Chord_result",chord_result1)
        new_chord_result, isAligned = Alignment.chord_alignment(chord_result1)
        song1_segments[idx].useable = isAligned
        
        #Squash the same beats together
        if isAligned:
            boolean_array = [False for i in range(len(new_chord_result))]
            for i in range(len(new_chord_result)):
                if i == 0 or new_chord_result[i] != new_chord_result[i-1]:
                    boolean_array[i] = True
            new_chords = []
            new_time = []
            for i in range(len(boolean_array)):
                if boolean_array[i]:
                    new_chords.append(new_chord_result[i])
                    new_time.append(time_1[i])
            new_chord_result = new_chords
            time_1 = new_time
            
        #Average time per beat
        avg_time_beat = 0
        for i in range(len(time_1) - 1):
            avg_time_beat += time_1[i+1] - time_1[i]
        avg_time_beat = avg_time_beat / len(time_1)
        print("Average time_per_chord", avg_time_beat)
        
        if not isAligned:
            print("Cannot align chord")
        
        song1_segments[idx].chords = new_chord_result
        song1_segments[idx].downbeats = time_1
        song1_segments[idx].average_time_per_beat = avg_time_beat
        
        
        beat_num1 = [1 for i in range(len(time_1))]
        
        dbt.print_and_save_chord(song1_name, time_1, song1_segments[idx].chords, str(idx) )
        dbt.print_and_save_downbeat(song1_name, time_1, beat_num1, str(idx))
        
        dump_data.append(song1_segments[idx].getJSON())    
    json.dump(dump_data, json_file, indent=4)
        

Loaded song_data with sr 44100 length 227.8560090702948
Segment info: Start:0 End:227.8560090702948 Duration: 227.8560090702948 Type: downbeat_length:0 chord_length:0 idx 0
length time_1 336
index 0 : 0.19 : 1.16 duration: 0.97 [(0., 1., 'B:min')]
index 1 : 1.16 : 2.17 duration: 1.01 [(0., 1.1, 'N')]
index 2 : 2.17 : 3.13 duration: 0.96 [(0., 1., 'B:min')]
index 3 : 3.13 : 3.97 duration: 0.8400000000000003 [(0., 0.9, 'N')]
index 4 : 3.97 : 4.84 duration: 0.8699999999999997 [(0., 0.9, 'N')]
index 5 : 4.84 : 5.79 duration: 0.9500000000000002 [(0., 1., 'B:min')]
index 6 : 5.79 : 6.73 duration: 0.9400000000000004 [(0., 1., 'N')]
index 7 : 6.73 : 7.59 duration: 0.8599999999999994 [(0., 0.9, 'N')]
index 8 : 7.59 : 8.46 duration: 0.870000000000001 [(0., 0.9, 'B:min')]
index 9 : 8.46 : 9.1 duration: 0.6399999999999988 [(0., 0.7, 'N')]
index 10 : 9.1 : 9.76 duration: 0.6600000000000001 [(0., 0.7, 'B:min')]
index 11 : 9.76 : 10.46 duration: 0.7000000000000011 [(0., 0.7, 'B:min')]
index 12 : 10.4

index 112 : 77.03 : 77.7 duration: 0.6700000000000017 [(0., 0.7, 'G:maj')]
index 113 : 77.7 : 78.37 duration: 0.6700000000000017 [(0., 0.7, 'G:maj')]
index 114 : 78.37 : 79.04 duration: 0.6700000000000017 [(0., 0.7, 'G:maj')]
index 115 : 79.04 : 79.73 duration: 0.6899999999999977 [(0., 0.7, 'G:maj')]
index 116 : 79.73 : 80.39 duration: 0.6599999999999966 [(0., 0.7, 'D:maj')]
index 117 : 80.39 : 81.07 duration: 0.6799999999999926 [(0., 0.7, 'D:maj')]
index 118 : 81.07 : 81.73 duration: 0.6600000000000108 [(0., 0.7, 'D:maj')]
index 119 : 81.73 : 82.42 duration: 0.6899999999999977 [(0., 0.7, 'D:maj')]
index 120 : 82.42 : 83.09 duration: 0.6700000000000017 [(0., 0.7, 'A:maj')]
index 121 : 83.09 : 83.77 duration: 0.6799999999999926 [(0., 0.7, 'A:maj')]
index 122 : 83.77 : 84.41 duration: 0.6400000000000006 [(0., 0.7, 'N')]
index 123 : 84.41 : 85.11 duration: 0.7000000000000028 [(0., 0.7, 'N')]
index 124 : 85.11 : 85.79 duration: 0.6800000000000068 [(0., 0.7, 'B:min')]
index 125 : 85.79 : 86

index 225 : 153.2 : 153.89 duration: 0.6899999999999977 [(0., 0.7, 'N')]
index 226 : 153.89 : 154.55 duration: 0.660000000000025 [(0., 0.7, 'G:maj')]
index 227 : 154.55 : 155.23 duration: 0.6799999999999784 [(0., 0.7, 'G:maj')]
index 228 : 155.23 : 155.89 duration: 0.6599999999999966 [(0., 0.7, 'G:maj')]
index 229 : 155.89 : 156.58 duration: 0.6900000000000261 [(0., 0.7, 'G:maj')]
index 230 : 156.58 : 157.24 duration: 0.6599999999999966 [(0., 0.7, 'D:maj')]
index 231 : 157.24 : 157.93 duration: 0.6899999999999977 [(0., 0.7, 'D:maj')]
index 232 : 157.93 : 158.58 duration: 0.6500000000000057 [(0., 0.7, 'D:maj')]
index 233 : 158.58 : 159.28 duration: 0.6999999999999886 [(0., 0.7, 'D:maj')]
index 234 : 159.28 : 159.95 duration: 0.6699999999999875 [(0., 0.7, 'A:maj')]
index 235 : 159.95 : 160.63 duration: 0.6800000000000068 [(0., 0.7, 'A:maj')]
index 236 : 160.63 : 161.27 duration: 0.6400000000000148 [(0., 0.7, 'N')]
index 237 : 161.27 : 161.96 duration: 0.6899999999999977 [(0., 0.7, 'N')]


index 334 : 226.67 : 227.36 duration: 0.6900000000000261 [(0., 0.7, 'N')]
index 335 : 227.36 : 227.8560090702948 duration: 0.49600907029477526 [(0., 0.5, 'N')]
Chord_result ['B:min', 'N', 'B:min', 'N', 'N', 'B:min', 'N', 'N', 'B:min', 'N', 'B:min', 'B:min', 'B:min', 'B:min', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'A:maj', 'A:maj', 'D:maj', 'N', 'B:min', 'B:min', 'B:min', 'N', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'N', 'D:maj', 'D:maj', 'D:maj', 'A:maj', 'A:maj', 'A:maj', 'N', 'B:min', 'B:min', 'B:min', 'B:min', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'A:maj', 'A:maj', 'A:maj', 'N', 'N', 'N', 'B:min', 'N', 'G:maj', 'N', 'N', 'N', 'D:maj', 'N', 'D:maj', 'N', 'A:maj', 'A:maj', 'D:maj', 'N', 'B:min', 'B:min', 'B:min', 'B:min', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'A:maj', 'A:maj', 'A:maj', 'N', 'N', 'N', 'N', 'B:min', 'B:min', 'B:min', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'D:maj', 'D:maj', 'D:m

In [5]:
sample,sr = dbt.load_and_resample(song1_name.replace(".wav", "") + ".wav")
downbeats = dbt.extract_downbeat(4,sample)
dbt.print_and_save_downbeat(song1_name, downbeats[:,0], downbeats[:,1])
# time_1, beat_num1 = dbt.filter_downbeat(downbeats)
time_1 = downbeats[:,0]
chord_dict, chord_result = dbt.get_chords_old(time_1, sample, sr)
dbt.print_and_save_chord(song1_name, downbeats[:,0], chord_result)



Saving downbeat of holiday.wav


ValueError: need at least one array to concatenate

In [6]:
print(downbeats)

[[  0.26   1.  ]
 [  0.69   2.  ]
 [  1.09   3.  ]
 ...
 [231.52   2.  ]
 [231.92   3.  ]
 [232.32   4.  ]]
